In [9]:
from sylliba.api.subnet_api import SubnetAPI
from sylliba.validator import reward_text, reward_speech
from sylliba.protocol import TranslateRequest
from modules.translation.data_models import TranslationRequest
from sylliba.utils.uids import get_top_miner_uids
import bittensor as bt
import math

In [2]:
request = {
    "input": "Hi How are you doing today?",
    "task_string": "text2speech",
    "source_language": "English",
    "target_language": "French"
}

In [3]:
translation_synapse = TranslateRequest(translation_request = TranslationRequest(data = request))

In [4]:
wallet = bt.wallet('sylliba', 'validator1', '~/.bittensor/wallets/')

In [6]:
subtensor = bt.subtensor(network='finney')
metagraph = subtensor.metagraph(netuid = 49)

miner_uids = await get_top_miner_uids(metagraph, wallet)
miner_axons = [metagraph.axons[uid] for uid in miner_uids]

In [7]:
subnet_api = SubnetAPI(wallet=wallet)

responses = await subnet_api(
    axons=miner_axons,
    synapse=translation_synapse,
    timeout=5
)

In [8]:
def process_validator_output(miner_response, sample_outputs, task_string):
    if task_string.endswith('text'):
        scores = [reward_text(miner_response, sample_output) for sample_output in sample_outputs]
    else:
        scores = [reward_speech(miner_response, sample_output) for sample_output in sample_outputs]
    return sum(scores) / len(scores)

In [ ]:
task_string = request['task_string']
results = []

for j in range(0, len(responses)):
    if responses[j].miner_response is not None:
        if task_string.endswith('speech'):
            miner_output_data = audio_decode(responses[j].miner_response)
        else:
            miner_output_data = responses[j].miner_response
                    
        results.append(
            float(process_validator_output(
                miner_output_data,
                sample_request['output'],
                task_string
            )) # 'numpy.float64' object cannot be interpreted as integer
        )

In [ ]:
best_response = responses[math.argmax(results)]